# READ ME
#### 1) Import the dictionaries
#### 2) Currently written for windows
#### 3) Subfolder name should be syntaxed like: mj_2-31_polymer_eis
#### 4) Input the alpha values for R+CPE fits, set to 1 if it's R + C
#### 4.1) THICKNESS IS CURRENTLY SET IN LINE 5
#### 5) Run all cells

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
# THIS IS THE MAC LINE
# eis = pd.read_table('/Users/Ratthew/OneDrive/lbl/capacitors/mj 2-31 polymer/mj_2-31_polymer_eis/n3-e_C08.txt', names = ['frequency', 'real', 'imaginary'])

In [4]:
# THIS IS THE PC LINE; notice the r!!
#eis = pd.read_table(r'C:\Users\YLiu\OneDrive\lbl\capacitors\mj 2-31 polymer\mj_2-31_polymer_eis\n3-e_C08.txt', names = ['frequency', 'real', 'imaginary'])
#eis['imaginary'] = np.abs(eis['imaginary'])
#eis.head(2)

In [5]:
# ok. theres the data. now we need to define the capacitance, relative permittivity and tan delta values
# easy one first...
def tan_d(real, imaginary):
    _tan_d = real/imaginary
    return _tan_d
#eis['tan d'] = tan_d(eis['real'], eis['imaginary'])  #COMMENTING THESE OUT MIGHT BREAK IT, NOT SURE

# capacitance = 1/imaginary*(2pi f)^alpha
def capacitance(imaginary, frequency, alpha):
    _capacitance = 1/(imaginary * np.power((2*np.pi*frequency), alpha))
    return _capacitance
# will have to input alpha eventually but for now just type it in manually 
#eis['capacitance'] = capacitance(eis['imaginary'], eis['frequency'], 1)

# krel = capacitance *  thickness   / e0 * dev area
dev_area  = 0.000003
E_naught = 8.85E-12
thickness = 4.08E-07
def krel(capval, thickness): #thickness also probably shouldnt be specified 
    _krel = ((capval * thickness)/(E_naught * dev_area))
    return _krel
#eis['K_rel'] = krel(eis['capacitance'], thickness)
#eis.head(2)

In [6]:
def CapDF(filename):
    subfolder = str('C:\\Users\\YLiu\\OneDrive\\lbl\\capacitors\\mj 2-31 polymer\\mj_2-31_polymer_eis\\')
    _eis = pd.read_table(subfolder + filename, names = ['frequency', 'real', 'imaginary'])
    _eis['imaginary'] = np.abs(_eis['imaginary'])
    _eis['tan d'] = tan_d(_eis['real'], _eis['imaginary'])
    _eis['capacitance'] = capacitance(_eis['imaginary'], _eis['frequency'], arfa[_counter]) #here is the alpha term!!!
    _eis['K_rel'] = krel(_eis['capacitance'], thickness)
    return _eis

In [7]:
# test = CapDF('n3-e_C08.txt')
# test.head(2)
# OK cool this function is now defined for any filename in the parent folder
# so now what do i want to do exactly, loop through the whole folder right? 
# and store each dataframe as the filename????

In [8]:
# now have to loop i through all files and plot them/aggregate into one big csv or somethign? 
# also should make the whole thing a function so i can call "process_data" on a cluster of files?

In [9]:
subfolder = input('subfolder?') #in this case it's like mj_2-31_polymer_eis or whatever
directory_to_see = os.getcwd()   +   "\\"   +  subfolder #this is all pc syntax
all_files = os.listdir(directory_to_see) #show me all files
text_list = []
for file1 in all_files:

    if file1.endswith('8.txt'): #as distinct from the fits which end in zfit.txt
        text_list.append("\\" + file1)

subfolder?mj_2-31_polymer_eis


In [10]:
text_list  #beast mode would be saving the alpha value somewhere in the filename

['\\a2-c_C08.txt',
 '\\a2-d_C08.txt',
 '\\a2-F_reallyF_C08.txt',
 '\\a2-g_C08.txt',
 '\\a2-h_C08.txt',
 '\\a3-c_C08.txt',
 '\\a3-g_C08.txt',
 '\\disconnected test_C08.txt',
 '\\n2-b_C08.txt',
 '\\n2-f_C08.txt',
 '\\n3-b_C08.txt',
 '\\n3-e_C08.txt',
 '\\short_C08.txt']

In [11]:
# MANUALLY INPUT ALPHAS HERE!!
# IN THE ORDER OF THE ABOVE FILE LIST!!!!!

arfa = [.986, .989, .988, .984, .990, .988, .992, 1, .977, .989, .993, .986, 1]


In [12]:
DF_holder_dict = {}
_counter = 0
for z in text_list:
    _name = 'DF' + z
    _name = _name.split('_')[0]
    _name = _name.replace("\\", "_")
    _name = _name.replace("-", "_")
    _name = _name.replace(" ", "_")
    DF_holder = str(_name)
    DF_holder_dict[DF_holder] = CapDF(z) #THANK YOU MAI DO!
    _counter = _counter + 1
# SO NOW WE HAVE A DICTIONARY OF ALL OUR DATA FRAMES WITH FAIRLY SENSIBLE NAMES
#DF_holder_dict["DF_a2_c"]

In [13]:
for key, value in DF_holder_dict.items() :
    print (key)

DF_a2_c
DF_a2_d
DF_a2_F
DF_a2_g
DF_a2_h
DF_a3_c
DF_a3_g
DF_disconnected_test
DF_n2_b
DF_n2_f
DF_n3_b
DF_n3_e
DF_short


In [14]:
newpath = directory_to_see + r'\processed_data'
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [15]:
for key, value in DF_holder_dict.items() :
    #print (key)
    DF_holder_dict[key].to_csv(newpath + '\\' + key +'.csv', sep = ',') 

In [ ]:
# cool. now you have a folder of all the proccessed device data